In [1]:
# This is added to get around some issues of Torch not loading models correctly (test on Mac OS X and Kubuntu Linux)
!pip install torch pillow hf_xet
!pip install torchcam
!pip install --upgrade huggingface-hub==0.26.2 transformers==4.46.1 tokenizers==0.20.1 diffusers==0.31.0


  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached diffusers-0.31.0-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
Using cached diffusers-0.31.0-py3-none-any.whl (2.9 MB)


## Local Inference on GPU
Model page: https://huggingface.co/prithivMLmods/deepfake-detector-model-v1

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/prithivMLmods/deepfake-detector-model-v1)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-classification", model="prithivMLmods/deepfake-detector-model-v1")
pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'Real', 'score': 0.788019061088562},
 {'label': 'Fake', 'score': 0.5715677738189697}]

In [3]:
import gradio as gr
from transformers import AutoImageProcessor, SiglipForImageClassification
from PIL import Image
import torch

# Load model and processor
model_name = "prithivMLmods/deepfake-detector-model-v1"
model = SiglipForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

# Updated label mapping
id2label = {
    "0": "fake",
    "1": "real"
}

def classify_image(image):
    image = Image.fromarray(image).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()

    prediction = {
        id2label[str(i)]: round(probs[i], 3) for i in range(len(probs))
    }

    return prediction


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
import os


def run_directory(filepath):
  for root, _, files in os.walk(filepath):
    for filename in files:  # loop through files in the current directory
          filepath = os.path.join(root, filename)
          image_array = np.asarray(Image.open(filepath))
          result = classify_image(image_array)
          if "real" in filepath:
            if "unaugmented" in filepath:
              if "sad" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/real_sad_unaugmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "happy" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/real_happy_unaugmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "neutral" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/real_neutral_unaugmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
            elif "augmented" in filepath:
              if "sad" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/real_sad_augmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "happy" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/real_happy_augmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "neutral" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/real_neutral_augmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
          elif "syn" in filepath:
            if "fingerprint" in filepath:
              if "sad" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/syn_sad_fingerprinted_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "happy" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/syn_happy_fingerprinted_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "neutral" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/syn_neutral_fingerprinted_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
            elif "unaugmented" in filepath:
              if "sad" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/syn_sad_unaugmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "happy" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/syn_happy_unaugmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()
              if "neutral" in filepath:
                file1 = open("/content/drive/MyDrive/Stable_diffusion_project_folder/deep_fake_detection_results/syn_neutral_unaugmented_results.txt", "a")
                file1.write(filepath + ": " + str(result) + "\n")
                file1.close()



In [6]:
run_directory("/content/drive/MyDrive/Stable_diffusion_project_folder/Images")

{'fake': 0.006, 'real': 0.994}
